[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kamalkraj/minGPT-TF/blob/master/play_math.ipynb)

In [42]:
#This is a 3 digs example and use all gpu with 100 iterations

In [43]:
! pip install fastprogress==0.2.3

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [44]:
import math
import numpy as np
import tensorflow as tf
from mingpt.model import GPT, GPTConfig

In [45]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [46]:
class AdditionDataset():
    """
    Returns addition problems of up to some number of digits in the inputs. Recall
    that all GPT cares about are sequences of integers, and completing them according to
    patterns in the data. Therefore, we have to somehow encode addition problems
    as a sequence of integers.
    
    The sum of two n-digit numbers gives a third up to (n+1)-digit number. So our
    encoding will simply be the n-digit first number, n-digit second number, 
    and (n+1)-digit result, all simply concatenated together. Because each addition
    problem is so structured, there is no need to bother the model with encoding
    +, =, or other tokens. Each possible sequence has the same length, and simply
    contains the raw digits of the addition problem.
    
    As a few examples, the 2-digit problems:
    - 85 + 50 = 135 becomes the sequence [8, 5, 5, 0, 1, 3, 5]
    - 6 + 39 = 45 becomes the sequence [0, 6, 3, 9, 0, 4, 5]
    etc.
    
    We will also only train GPT on the final (n+1)-digits because the first
    two n-digits are always assumed to be given. So when we give GPT an exam later,
    we will e.g. feed it the sequence [0, 6, 3, 9], which encodes that we'd like
    to add 6 + 39, and hope that the model completes the integer sequence with [0, 4, 5]
    in 3 sequential steps.
    
    fun exercise: does it help if the result is asked to be produced in reverse order?
    """

    def __init__(self, ndigit, split):
        self.split = split # train/test
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        # +1 due to potential carry overflow, but then -1 because very last digit doesn't plug back
        self.block_size = ndigit + ndigit + ndigit + 1 - 1
        
        # split up all addition problems into either training data or test data
        num = (10**self.ndigit)**2 # total number of possible combinations
        r = np.random.RandomState(1337) # make deterministic
        perm = r.permutation(num)
        num_test = min(int(num*0.2), 1000) # 20% of the whole dataset, or only up to 1000
        self.ixes = perm[:num_test] if split == 'test' else perm[num_test:]

    def __len__(self):
        return self.ixes.size

    def __iter__(self):
        # given a problem index idx, first recover the associated a + b
        #Save the train and test data to file
        #fileName = './dataset/train_set_math_3dig.txt' if self.split == 'test' else './dataset/test_set_math_3dig.txt'
        #file = open(fileName, 'a')
        
        for idx in range(self.__len__()):
            idx = self.ixes[idx]
            nd = 10**self.ndigit
            a = idx // nd
            b = idx %  nd
            c = a + b
            #print('idx='+str(idx) +' nd= ' + str(nd) + ' a=' + str(a) + ' b=' +str(b) + ' c=' +str(c))
            #idx=9927 nd= 100 a=99 b=27 c=126
            render = f'%0{self.ndigit}d%0{self.ndigit}d%0{self.ndigit+1}d' % (a,b,c) # e.g. 03+25=28 becomes "0325028"    
            #print('render= ' + render)
            #render= 9927126
            #write the dataset to file
            #file.write(render+'\n')
            
            dix = [int(s) for s in render] # convert each character to its token index
            #print('dix= ' + str(dix))
            #dix= [9, 9, 2, 7, 1, 2, 6]
            
            # x will be input to GPT and y will be the associated expected outputs
            x = dix[:-1]
            #print('x = ' + str(x))
            #x = [9, 9, 2, 7, 1, 2]
            
            y = dix[1:] # predict the next token in the sequence
            #print('y = ' + str(y))
            #y = [9, 2, 7, 1, 2, 6]
            
            y[:self.ndigit*2-1] = [-1] * (self.ndigit*2-1) # we will only train in the output locations. -100 will mask loss to zero
            #print('y = ' + str(y))
            #y = [-1, -1, -1, 1, 2, 6]
            
            x = tf.convert_to_tensor(x,dtype=tf.int32)
            y = tf.convert_to_tensor(y,dtype=tf.int32)
            #print('tf x = ' + str(x))
            #print('tf y = ' + str(y))
            #tf x = tf.Tensor([9 9 2 7 1 2], shape=(6,), dtype=int32)
            #tf y = tf.Tensor([-1 -1 -1  1  2  6], shape=(6,), dtype=int32)
            
            yield x, y
            
        #file.close()    
    __call__ = __iter__

In [47]:
# create a dataset for e.g. 2-digit addition
ndigit = 2
train_dataset_gen = AdditionDataset(ndigit=ndigit, split='train')
test_dataset_gen = AdditionDataset(ndigit=ndigit, split='test')

In [48]:
train_dataset = tf.data.Dataset.from_generator(train_dataset_gen,(tf.int32,tf.int32))
test_dataset = tf.data.Dataset.from_generator(test_dataset_gen,(tf.int32,tf.int32))

In [49]:
# initialize a baby GPT model
mconf = GPTConfig(train_dataset_gen.vocab_size, train_dataset_gen.block_size, 
                  n_layer=2, n_head=4, n_embd=128)
# model = GPT(mconf)

In [50]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=50, batch_size=1024, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=50*len(train_dataset_gen)*(ndigit+1),
                      num_workers=4)
trainer = Trainer(GPT, mconf, train_dataset, len(train_dataset_gen), test_dataset, len(test_dataset_gen), tconf, device='GPU:1')

config.vocab_size=10,config.n_embd=128
config.block_size=6,config.n_embd=128
config.embd_pdrop=0.1
config.n_embd=128,config.n_head=4,config.attn_pdrop=0.1,config.resid_pdrop=0.1,config.n_layer=2
EncoderLayer d_model=128,num_heads=4
EncoderLayer d_model=128,num_heads=4


In [51]:
#train the first time and save checkpoints
trainer.train()

epoch 1: train loss 3.92096. lr 5.994512e-04
epoch 1: test loss 2.04901.
The gpt train model weight is saved to checkpoints:./checkpoings/ckpt_math_3dig/minigpt.ckpt


In [52]:
trainer.display_model()

Model: "gpt_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     multiple                  1280      
                                                                 
 dropout_21 (Dropout)        multiple                  0         
                                                                 
 encoder_layer_6 (EncoderLay  multiple                 198272    
 er)                                                             
                                                                 
 encoder_layer_7 (EncoderLay  multiple                 198272    
 er)                                                             
                                                                 
 layer_normalization_19 (Lay  multiple                 256       
 erNormalization)                                                
                                                             

In [53]:
from mingpt.utils import sample

In [54]:
def give_exam(dataset, batch_size=32, max_batches=-1, printResult=False):
    
    results = []
    
    loader = dataset.batch(batch_size)
    for b, (x, y) in enumerate(loader):
        #print('b='+str(b))
        #print('x='+str(x))
        #print('y='+str(y))
        #b=0
        #x=tf.Tensor([[9 9 2 7 1 2]], shape=(1, 6), dtype=int32)
        #y=tf.Tensor([[-1 -1 -1  1  2  6]], shape=(1, 6), dtype=int32)

        d1d2 = x[:, :ndigit*2]
        #print('d1d2=' +str(d1d2))
        #d1d2=tf.Tensor([[9 9 2 7]], shape=(1, 4), dtype=int32)

        d1d2d3 = sample(trainer.model, d1d2, ndigit+1)
        #print('d1d2d3=' +str(d1d2d3))
        #d1d2d3=tf.Tensor([[9 9 2 7 1 1 1]], shape=(1, 7), dtype=int32)

        d3 = d1d2d3[:, -(ndigit+1):]
        #print('d3=' +str(d3))
        #d3=tf.Tensor([[1 1 1]], shape=(1, 3), dtype=int32)

        factors = tf.convert_to_tensor([[10**i for i in range(ndigit+1)][::-1]])
        #print('factors=' +str(factors))
        #factors=tf.Tensor([[100  10   1]], shape=(1, 3), dtype=int32)
        
        # decode the integers from individual digits
        d1i = tf.reduce_sum((d1d2[:,:ndigit] * factors[:,1:]),axis=1)
        #print('d1i='+str(d1i))
        #d1i=tf.Tensor([99], shape=(1,), dtype=int32)

        d2i = tf.reduce_sum((d1d2[:,ndigit:ndigit*2] * factors[:,1:]),axis=1)
        #print('d2i='+str(d2i))
        #d2i=tf.Tensor([27], shape=(1,), dtype=int32)
        
        d3i_pred = tf.reduce_sum((d3 * factors),axis=1)
        #print('d3i_pred='+str(d3i_pred))
        #d3i_pred=tf.Tensor([111], shape=(1,), dtype=int32)
        
        d3i_gt = d1i + d2i
        correct = (d3i_pred == d3i_gt) # Software 1.0 vs. Software 2.0 fight RIGHT on this line, lol
        for i in range(x.shape[0]):
            results.append(int(correct[i]))
            judge = 'YEP!!!' if correct[i] else 'NOPE'
            if not correct[i]:
                print("GPT claims that %03d + %03d = %03d (gt is %03d; %s)========" 
                      % (d1i[i], d2i[i], d3i_pred[i], d3i_gt[i], judge))
            elif printResult:
                print("GPT claims that %03d + %03d = %03d (gt is %03d; %s)" 
                      % (d1i[i], d2i[i], d3i_pred[i], d3i_gt[i], judge))                
        
        if max_batches >= 0 and b+1 >= max_batches:
            break

    print("final score: %d/%d = %.2f%% correct" % (np.sum(results), len(results), 100*np.mean(results)))

In [55]:
#if train already and load the weights from checkpoint
#trainer.load_checkpoints()

In [56]:
give_exam(train_dataset,batch_size=128,max_batches=-1,printResult=False)

GPT claims that 034 + 055 = 079 (gt is 089; NOPE)========
GPT claims that 018 + 027 = 044 (gt is 045; NOPE)========
GPT claims that 086 + 028 = 103 (gt is 114; NOPE)========
GPT claims that 086 + 030 = 127 (gt is 116; NOPE)========
GPT claims that 047 + 033 = 079 (gt is 080; NOPE)========
GPT claims that 032 + 002 = 044 (gt is 034; NOPE)========
GPT claims that 044 + 072 = 126 (gt is 116; NOPE)========
GPT claims that 066 + 070 = 146 (gt is 136; NOPE)========
GPT claims that 071 + 035 = 116 (gt is 106; NOPE)========
GPT claims that 038 + 090 = 129 (gt is 128; NOPE)========
GPT claims that 022 + 086 = 107 (gt is 108; NOPE)========
GPT claims that 081 + 051 = 140 (gt is 132; NOPE)========
GPT claims that 001 + 066 = 077 (gt is 067; NOPE)========
GPT claims that 093 + 098 = 172 (gt is 191; NOPE)========
GPT claims that 017 + 068 = 074 (gt is 085; NOPE)========
GPT claims that 070 + 065 = 146 (gt is 135; NOPE)========
GPT claims that 066 + 066 = 131 (gt is 132; NOPE)========
GPT claims tha

GPT claims that 086 + 066 = 141 (gt is 152; NOPE)========
GPT claims that 077 + 005 = 072 (gt is 082; NOPE)========
GPT claims that 083 + 035 = 119 (gt is 118; NOPE)========
GPT claims that 007 + 051 = 067 (gt is 058; NOPE)========
GPT claims that 098 + 083 = 171 (gt is 181; NOPE)========
GPT claims that 057 + 008 = 064 (gt is 065; NOPE)========
GPT claims that 028 + 083 = 101 (gt is 111; NOPE)========
GPT claims that 082 + 093 = 165 (gt is 175; NOPE)========
GPT claims that 059 + 042 = 442 (gt is 101; NOPE)========
GPT claims that 033 + 030 = 064 (gt is 063; NOPE)========
GPT claims that 080 + 030 = 120 (gt is 110; NOPE)========
GPT claims that 010 + 074 = 074 (gt is 084; NOPE)========
GPT claims that 073 + 020 = 104 (gt is 093; NOPE)========
GPT claims that 089 + 005 = 074 (gt is 094; NOPE)========
GPT claims that 024 + 011 = 045 (gt is 035; NOPE)========
GPT claims that 065 + 029 = 074 (gt is 094; NOPE)========
GPT claims that 047 + 098 = 144 (gt is 145; NOPE)========
GPT claims tha

GPT claims that 074 + 025 = 100 (gt is 099; NOPE)========
GPT claims that 043 + 065 = 109 (gt is 108; NOPE)========
GPT claims that 071 + 050 = 131 (gt is 121; NOPE)========
GPT claims that 019 + 060 = 070 (gt is 079; NOPE)========
GPT claims that 046 + 091 = 147 (gt is 137; NOPE)========
GPT claims that 009 + 091 = 070 (gt is 100; NOPE)========
GPT claims that 099 + 083 = 162 (gt is 182; NOPE)========
GPT claims that 029 + 088 = 106 (gt is 117; NOPE)========
GPT claims that 011 + 084 = 105 (gt is 095; NOPE)========
GPT claims that 007 + 058 = 064 (gt is 065; NOPE)========
GPT claims that 093 + 070 = 164 (gt is 163; NOPE)========
GPT claims that 068 + 003 = 062 (gt is 071; NOPE)========
GPT claims that 063 + 000 = 064 (gt is 063; NOPE)========
GPT claims that 012 + 086 = 107 (gt is 098; NOPE)========
GPT claims that 046 + 039 = 074 (gt is 085; NOPE)========
GPT claims that 057 + 000 = 067 (gt is 057; NOPE)========
GPT claims that 000 + 064 = 074 (gt is 064; NOPE)========
GPT claims tha

GPT claims that 085 + 042 = 137 (gt is 127; NOPE)========
GPT claims that 029 + 085 = 103 (gt is 114; NOPE)========
GPT claims that 077 + 031 = 119 (gt is 108; NOPE)========
GPT claims that 027 + 054 = 071 (gt is 081; NOPE)========
GPT claims that 053 + 010 = 064 (gt is 063; NOPE)========
GPT claims that 093 + 045 = 149 (gt is 138; NOPE)========
GPT claims that 045 + 036 = 071 (gt is 081; NOPE)========
GPT claims that 003 + 025 = 037 (gt is 028; NOPE)========
GPT claims that 011 + 026 = 047 (gt is 037; NOPE)========
GPT claims that 059 + 065 = 114 (gt is 124; NOPE)========
GPT claims that 027 + 008 = 044 (gt is 035; NOPE)========
GPT claims that 017 + 058 = 064 (gt is 075; NOPE)========
GPT claims that 081 + 009 = 071 (gt is 090; NOPE)========
GPT claims that 002 + 041 = 044 (gt is 043; NOPE)========
GPT claims that 084 + 047 = 121 (gt is 131; NOPE)========
GPT claims that 016 + 049 = 064 (gt is 065; NOPE)========
GPT claims that 006 + 077 = 073 (gt is 083; NOPE)========
GPT claims tha

GPT claims that 041 + 082 = 124 (gt is 123; NOPE)========
GPT claims that 003 + 008 = 022 (gt is 011; NOPE)========
GPT claims that 097 + 095 = 172 (gt is 192; NOPE)========
GPT claims that 000 + 023 = 033 (gt is 023; NOPE)========
GPT claims that 075 + 030 = 106 (gt is 105; NOPE)========
GPT claims that 084 + 009 = 073 (gt is 093; NOPE)========
GPT claims that 044 + 009 = 052 (gt is 053; NOPE)========
GPT claims that 043 + 073 = 110 (gt is 116; NOPE)========
GPT claims that 037 + 077 = 104 (gt is 114; NOPE)========
GPT claims that 036 + 019 = 044 (gt is 055; NOPE)========
GPT claims that 067 + 069 = 146 (gt is 136; NOPE)========
GPT claims that 041 + 044 = 075 (gt is 085; NOPE)========
GPT claims that 097 + 071 = 169 (gt is 168; NOPE)========
GPT claims that 073 + 022 = 105 (gt is 095; NOPE)========
GPT claims that 004 + 046 = 049 (gt is 050; NOPE)========
GPT claims that 061 + 025 = 076 (gt is 086; NOPE)========
GPT claims that 005 + 063 = 077 (gt is 068; NOPE)========
GPT claims tha

...

GPT claims that 014 + 095 = 100 (gt is 109; NOPE)========
GPT claims that 075 + 059 = 133 (gt is 134; NOPE)========
GPT claims that 089 + 062 = 141 (gt is 151; NOPE)========
GPT claims that 082 + 051 = 144 (gt is 133; NOPE)========
GPT claims that 029 + 014 = 042 (gt is 043; NOPE)========
GPT claims that 052 + 026 = 077 (gt is 078; NOPE)========
GPT claims that 056 + 000 = 066 (gt is 056; NOPE)========
GPT claims that 092 + 059 = 142 (gt is 151; NOPE)========
GPT claims that 050 + 085 = 145 (gt is 135; NOPE)========
GPT claims that 020 + 012 = 042 (gt is 032; NOPE)========
GPT claims that 041 + 062 = 104 (gt is 103; NOPE)========
GPT claims that 052 + 088 = 133 (gt is 140; NOPE)========
GPT claims that 008 + 031 = 049 (gt is 039; NOPE)========
GPT claims that 055 + 013 = 067 (gt is 068; NOPE)========
GPT claims that 069 + 023 = 072 (gt is 092; NOPE)========
GPT claims that 056 + 081 = 147 (gt is 137; NOPE)========
GPT claims that 096 + 088 = 174 (gt is 184; NOPE)========
GPT claims tha

GPT claims that 068 + 019 = 076 (gt is 087; NOPE)========
GPT claims that 005 + 014 = 029 (gt is 019; NOPE)========
GPT claims that 089 + 022 = 101 (gt is 111; NOPE)========
GPT claims that 096 + 078 = 164 (gt is 174; NOPE)========
GPT claims that 010 + 005 = 025 (gt is 015; NOPE)========
GPT claims that 031 + 099 = 121 (gt is 130; NOPE)========
GPT claims that 074 + 004 = 070 (gt is 078; NOPE)========
GPT claims that 032 + 015 = 046 (gt is 047; NOPE)========
GPT claims that 049 + 008 = 056 (gt is 057; NOPE)========
GPT claims that 054 + 048 = 092 (gt is 102; NOPE)========
GPT claims that 065 + 022 = 077 (gt is 087; NOPE)========
GPT claims that 049 + 065 = 104 (gt is 114; NOPE)========
GPT claims that 029 + 087 = 106 (gt is 116; NOPE)========
GPT claims that 034 + 044 = 070 (gt is 078; NOPE)========
GPT claims that 072 + 086 = 159 (gt is 158; NOPE)========
GPT claims that 064 + 021 = 075 (gt is 085; NOPE)========
GPT claims that 017 + 055 = 062 (gt is 072; NOPE)========
GPT claims tha

GPT claims that 025 + 025 = 040 (gt is 050; NOPE)========
GPT claims that 027 + 009 = 046 (gt is 036; NOPE)========
GPT claims that 049 + 068 = 106 (gt is 117; NOPE)========
GPT claims that 051 + 001 = 060 (gt is 052; NOPE)========
GPT claims that 037 + 000 = 047 (gt is 037; NOPE)========
GPT claims that 049 + 042 = 322 (gt is 091; NOPE)========
GPT claims that 042 + 092 = 144 (gt is 134; NOPE)========
GPT claims that 048 + 071 = 129 (gt is 119; NOPE)========
GPT claims that 074 + 086 = 159 (gt is 160; NOPE)========
GPT claims that 045 + 088 = 132 (gt is 133; NOPE)========
GPT claims that 037 + 047 = 074 (gt is 084; NOPE)========
GPT claims that 044 + 039 = 073 (gt is 083; NOPE)========
GPT claims that 040 + 083 = 124 (gt is 123; NOPE)========
GPT claims that 072 + 044 = 126 (gt is 116; NOPE)========
GPT claims that 093 + 057 = 140 (gt is 150; NOPE)========
GPT claims that 051 + 078 = 139 (gt is 129; NOPE)========
GPT claims that 016 + 064 = 079 (gt is 080; NOPE)========
GPT claims tha

GPT claims that 020 + 091 = 121 (gt is 111; NOPE)========
GPT claims that 039 + 069 = 107 (gt is 108; NOPE)========
GPT claims that 002 + 035 = 046 (gt is 037; NOPE)========
GPT claims that 000 + 060 = 070 (gt is 060; NOPE)========
GPT claims that 000 + 010 = 020 (gt is 010; NOPE)========
GPT claims that 041 + 070 = 121 (gt is 111; NOPE)========
GPT claims that 035 + 028 = 062 (gt is 063; NOPE)========
GPT claims that 097 + 028 = 124 (gt is 125; NOPE)========
GPT claims that 070 + 044 = 124 (gt is 114; NOPE)========
GPT claims that 096 + 079 = 164 (gt is 175; NOPE)========
GPT claims that 081 + 053 = 144 (gt is 134; NOPE)========
GPT claims that 076 + 065 = 140 (gt is 141; NOPE)========
GPT claims that 091 + 097 = 179 (gt is 188; NOPE)========
GPT claims that 046 + 006 = 051 (gt is 052; NOPE)========
GPT claims that 058 + 098 = 146 (gt is 156; NOPE)========
GPT claims that 077 + 001 = 077 (gt is 078; NOPE)========
GPT claims that 011 + 091 = 111 (gt is 102; NOPE)========
GPT claims tha

GPT claims that 015 + 032 = 046 (gt is 047; NOPE)========
GPT claims that 047 + 069 = 106 (gt is 116; NOPE)========
GPT claims that 043 + 097 = 131 (gt is 140; NOPE)========
GPT claims that 079 + 086 = 164 (gt is 165; NOPE)========
GPT claims that 025 + 045 = 060 (gt is 070; NOPE)========
GPT claims that 089 + 078 = 166 (gt is 167; NOPE)========
GPT claims that 060 + 020 = 070 (gt is 080; NOPE)========
GPT claims that 008 + 039 = 046 (gt is 047; NOPE)========
GPT claims that 087 + 012 = 109 (gt is 099; NOPE)========
GPT claims that 089 + 000 = 079 (gt is 089; NOPE)========
GPT claims that 069 + 038 = 106 (gt is 107; NOPE)========
GPT claims that 058 + 079 = 136 (gt is 137; NOPE)========
GPT claims that 083 + 088 = 161 (gt is 171; NOPE)========
GPT claims that 009 + 002 = 022 (gt is 011; NOPE)========
GPT claims that 047 + 030 = 067 (gt is 077; NOPE)========
GPT claims that 036 + 055 = 070 (gt is 091; NOPE)========
GPT claims that 004 + 004 = 020 (gt is 008; NOPE)========
GPT claims tha

In [57]:
give_exam(test_dataset,batch_size=128,max_batches=-1,printResult=True)

GPT claims that 099 + 027 = 126 (gt is 126; YEP!!!)
GPT claims that 032 + 045 = 077 (gt is 077; YEP!!!)
GPT claims that 023 + 003 = 035 (gt is 026; NOPE)========
GPT claims that 077 + 007 = 074 (gt is 084; NOPE)========
GPT claims that 088 + 020 = 109 (gt is 108; NOPE)========
GPT claims that 062 + 006 = 067 (gt is 068; NOPE)========
GPT claims that 068 + 095 = 152 (gt is 163; NOPE)========
GPT claims that 059 + 024 = 093 (gt is 083; NOPE)========
GPT claims that 064 + 023 = 077 (gt is 087; NOPE)========
GPT claims that 041 + 095 = 146 (gt is 136; NOPE)========
GPT claims that 015 + 089 = 104 (gt is 104; YEP!!!)
GPT claims that 047 + 065 = 101 (gt is 112; NOPE)========
GPT claims that 072 + 050 = 132 (gt is 122; NOPE)========
GPT claims that 046 + 045 = 090 (gt is 091; NOPE)========
GPT claims that 087 + 006 = 073 (gt is 093; NOPE)========
GPT claims that 017 + 061 = 077 (gt is 078; NOPE)========
GPT claims that 003 + 065 = 079 (gt is 068; NOPE)========
GPT claims that 010 + 049 = 060 

GPT claims that 033 + 036 = 069 (gt is 069; YEP!!!)
GPT claims that 088 + 040 = 139 (gt is 128; NOPE)========
GPT claims that 054 + 066 = 110 (gt is 120; NOPE)========
GPT claims that 022 + 083 = 105 (gt is 105; YEP!!!)
GPT claims that 049 + 075 = 124 (gt is 124; YEP!!!)
GPT claims that 066 + 025 = 070 (gt is 091; NOPE)========
GPT claims that 050 + 032 = 072 (gt is 082; NOPE)========
GPT claims that 003 + 017 = 021 (gt is 020; NOPE)========
GPT claims that 051 + 053 = 104 (gt is 104; YEP!!!)
GPT claims that 003 + 009 = 022 (gt is 012; NOPE)========
GPT claims that 035 + 054 = 079 (gt is 089; NOPE)========
GPT claims that 090 + 072 = 162 (gt is 162; YEP!!!)
GPT claims that 057 + 076 = 133 (gt is 133; YEP!!!)
GPT claims that 060 + 024 = 074 (gt is 084; NOPE)========
GPT claims that 099 + 067 = 166 (gt is 166; YEP!!!)
GPT claims that 004 + 077 = 072 (gt is 081; NOPE)========
GPT claims that 091 + 002 = 094 (gt is 093; NOPE)========
GPT claims that 057 + 079 = 136 (gt is 136; YEP!!!)
GPT 

GPT claims that 077 + 006 = 073 (gt is 083; NOPE)========
GPT claims that 005 + 079 = 074 (gt is 084; NOPE)========
GPT claims that 032 + 058 = 072 (gt is 090; NOPE)========
GPT claims that 010 + 045 = 065 (gt is 055; NOPE)========
GPT claims that 009 + 007 = 026 (gt is 016; NOPE)========
GPT claims that 006 + 052 = 067 (gt is 058; NOPE)========
GPT claims that 060 + 068 = 139 (gt is 128; NOPE)========
GPT claims that 073 + 005 = 079 (gt is 078; NOPE)========
GPT claims that 089 + 021 = 101 (gt is 110; NOPE)========
GPT claims that 018 + 066 = 073 (gt is 084; NOPE)========
GPT claims that 025 + 073 = 107 (gt is 098; NOPE)========
GPT claims that 035 + 008 = 042 (gt is 043; NOPE)========
GPT claims that 065 + 042 = 107 (gt is 107; YEP!!!)
GPT claims that 070 + 062 = 142 (gt is 132; NOPE)========
GPT claims that 017 + 018 = 034 (gt is 035; NOPE)========
GPT claims that 073 + 034 = 107 (gt is 107; YEP!!!)
GPT claims that 042 + 077 = 129 (gt is 119; NOPE)========
GPT claims that 073 + 059 

GPT claims that 021 + 036 = 067 (gt is 057; NOPE)========
GPT claims that 017 + 085 = 101 (gt is 102; NOPE)========
GPT claims that 022 + 017 = 049 (gt is 039; NOPE)========
GPT claims that 045 + 079 = 124 (gt is 124; YEP!!!)
GPT claims that 066 + 042 = 109 (gt is 108; NOPE)========
GPT claims that 059 + 078 = 136 (gt is 137; NOPE)========
GPT claims that 089 + 014 = 103 (gt is 103; YEP!!!)
GPT claims that 027 + 081 = 109 (gt is 108; NOPE)========
GPT claims that 014 + 016 = 030 (gt is 030; YEP!!!)
GPT claims that 076 + 081 = 157 (gt is 157; YEP!!!)
GPT claims that 069 + 051 = 111 (gt is 120; NOPE)========
GPT claims that 083 + 048 = 121 (gt is 131; NOPE)========
GPT claims that 086 + 008 = 074 (gt is 094; NOPE)========
GPT claims that 032 + 080 = 112 (gt is 112; YEP!!!)
GPT claims that 028 + 058 = 076 (gt is 086; NOPE)========
GPT claims that 096 + 097 = 173 (gt is 193; NOPE)========
GPT claims that 088 + 034 = 112 (gt is 122; NOPE)========
GPT claims that 075 + 091 = 166 (gt is 166; 

GPT claims that 027 + 029 = 046 (gt is 056; NOPE)========
GPT claims that 096 + 098 = 174 (gt is 194; NOPE)========
GPT claims that 079 + 034 = 113 (gt is 113; YEP!!!)
GPT claims that 048 + 057 = 104 (gt is 105; NOPE)========
GPT claims that 027 + 072 = 109 (gt is 099; NOPE)========
GPT claims that 074 + 091 = 165 (gt is 165; YEP!!!)
GPT claims that 011 + 041 = 060 (gt is 052; NOPE)========
GPT claims that 080 + 050 = 140 (gt is 130; NOPE)========
GPT claims that 066 + 097 = 153 (gt is 163; NOPE)========
GPT claims that 036 + 023 = 069 (gt is 059; NOPE)========
GPT claims that 050 + 059 = 109 (gt is 109; YEP!!!)
GPT claims that 068 + 044 = 102 (gt is 112; NOPE)========
GPT claims that 068 + 089 = 146 (gt is 157; NOPE)========
GPT claims that 003 + 018 = 022 (gt is 021; NOPE)========
GPT claims that 024 + 023 = 047 (gt is 047; YEP!!!)
GPT claims that 082 + 017 = 109 (gt is 099; NOPE)========
GPT claims that 053 + 099 = 142 (gt is 152; NOPE)========
GPT claims that 062 + 066 = 139 (gt is

GPT claims that 087 + 059 = 146 (gt is 146; YEP!!!)
GPT claims that 005 + 017 = 022 (gt is 022; YEP!!!)
GPT claims that 032 + 070 = 102 (gt is 102; YEP!!!)
GPT claims that 038 + 006 = 043 (gt is 044; NOPE)========
GPT claims that 026 + 001 = 037 (gt is 027; NOPE)========
GPT claims that 069 + 043 = 102 (gt is 112; NOPE)========
GPT claims that 067 + 048 = 114 (gt is 115; NOPE)========
GPT claims that 074 + 016 = 070 (gt is 090; NOPE)========
GPT claims that 065 + 071 = 146 (gt is 136; NOPE)========
GPT claims that 045 + 057 = 102 (gt is 102; YEP!!!)
GPT claims that 003 + 064 = 077 (gt is 067; NOPE)========
GPT claims that 076 + 003 = 079 (gt is 079; YEP!!!)
GPT claims that 036 + 068 = 103 (gt is 104; NOPE)========
GPT claims that 021 + 045 = 066 (gt is 066; YEP!!!)
GPT claims that 084 + 089 = 162 (gt is 173; NOPE)========
GPT claims that 054 + 074 = 120 (gt is 128; NOPE)========
GPT claims that 037 + 023 = 069 (gt is 060; NOPE)========
GPT claims that 041 + 086 = 127 (gt is 127; YEP!!!